# Vgg16 entrenada en spyder

### Carga del modelo entrenado:

A continuación se muestra el código de la cnn entrenada, basándose en la red pre-entrenada vgg16, congelando todas las capas excepto la top layer incluida para distinguir entre tres clases diferentes:

In [ ]:
import sys
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers import  Convolution2D, MaxPooling2D, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
from keras import applications

K.clear_session()
np.random.seed(0)

def modelo():
    vgg=applications.vgg16.VGG16()
    cnn=Sequential()
    for capa in vgg.layers:
        cnn.add(capa)
    cnn.layers.pop()
    for layer in cnn.layers:
        layer.trainable=False
    cnn.add(Dense(3,activation='softmax'))
    return cnn

#Se almacenan en variables los directorios en los que se encuentran las imágenes
data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'

#Parámetros importantes:
epocas=20
longitud, altura = 224, 224
batch_size = 32 #Imágenes a procesar en cada paso
pasos = 100
validation_steps = 100 #Imágenes de validación que se pasan al final de cada época
clases = 3
lr = 0.0004 #Learning rate

###Procesamiento del conjunto de entrenamieto:
#Se aplican mecanismos de data argumentation
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255, 
    shear_range=0.2, #Inclina las imágenes
    zoom_range=0.2, #Zoom a algunas imágenes
    horizontal_flip=True) #Invierte imágenes para distinguir direcionalidad

###Procesamiento del conjunto de validación:
#No es necesario inclinar, hacer zoom ni invertir las imágenes.
test_datagen = ImageDataGenerator(rescale=1. / 255)

###Generación del conjunto de entrenamieto:
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical') #Se busca una clasificación categórica

###Generación del conjunto de validación:
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

model=modelo()
model.summary()
model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])

model.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

print(entrenamiento_generador.class_indices)

#score = model.evaluate_generator(validacion_generador, steps=20, verbose=1)
#print('Test accuracy:', score[1])

dir = './vgg16_largo/'
if not os.path.exists(dir):
    os.mkdir(dir)
model.save('./vgg16_largo/modelo_vgg16_largo.h5')#Se guarda la estructura de la cnn
model.save_weights('./vgg16_largo/pesos_vgg16_largo.h5')#Se guardan los pesos de la cnn


La arquitectura y pesos de esto modelo han sido guardados en el directorio './vgg16_largo/'. Se cargan la red para realizar predicciones:

In [7]:
carga_modelo = './vgg16_largo/modelo_vgg16_largo.h5'
pesos_modelo = './vgg16_largo/pesos_vgg16_largo.h5'
cnn = load_model(carga_modelo)
cnn.load_weights(pesos_modelo)
cnn.summary()

ValueError: You are trying to load a weight file containing 17 layers into a model with 0 layers

Se prueba generando la arquitectura de la red de nuevo y cargar los pesos guardados en el directorio './vgg16_largo/pesos_vgg16_largo.h5'

In [8]:
vgg=applications.vgg16.VGG16()
cnn=Sequential()
for capa in vgg.layers:
    cnn.add(capa)
cnn.layers.pop()
for layer in cnn.layers:
    layer.trainable=False
cnn.add(Dense(3,activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

Ahora si, cargamos los pesos obtenidos con el entrenamiento previo de la red:

In [10]:
pesos_modelo = './vgg16_largo/pesos_vgg16_largo.h5'
cnn.load_weights(pesos_modelo)

### Resultados entrenamiento:

Evaluando el modelo se obtubieron los siguientes resultados:

In [ ]:
score = model.evaluate_generator(validacion_generador, steps=pasos, verbose=1)
print('Test accuracy:', score[1])
print('Loss:', score[0])

100/100 [==============================] - 2023s 20s/step

Test accuracy: 0.5018808776121528

Loss: 0.7800169631978935

### Predicciones:

NOTA: para conocer el índice que se corresponde con cada clase, se usa el siguiente comando:

In [ ]:
print(entrenamiento_generador.class_indices)

{'gato': 0, 'gorila': 1, 'perro': 2}

In [14]:
###Función predicción:
def predict(file):
  x = load_img(file, target_size=(224, 224))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x)
  print('Probabilidades:', array)  
  result = array[0]
  answer = np.argmax(result) #Devuelve el índice en el que se encuentra el valor máximo
  print(answer)  
  if answer == 0:
    print("pred: Gato")
  elif answer == 1:
    print("pred: Gorila")
  elif answer == 2:
    print("pred: Perro")
  return answer

predict('cat.4.jpg')

Probabilidades: [[0.4848556  0.07351075 0.44163367]]
0
pred: Gato


0

Para este caso, se cumple la predicción, y la imagen 'cat.4.jpg' efectivamente es un gato. En cambio si realizamos más predicciones vemos como muchas no son correctas, poniéndose de manifiesto el porcentaje de acierto (Test accuracy: 0.5018808776121528) obtenido en la evaluación del modelo.